In [13]:
import pandas as pd

df = pd.read_csv("/Users/ijunhui/Downloads/ubion/프로젝트2/1_기업선정/정상기업/2017정상기업전처리.csv")

list_a = df['회사명'].unique().tolist()

com_list = list_a


In [102]:
com_list.index('모베이스전자')


206

In [104]:
# step1.프로젝트에 필요한 패키지 불러오기
from asyncio import start_server
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import math
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException


# step2.로그인 정보 및 검색할 회사 미리 정의, 해당 회사의 리뷰 끝 페이지도 정의
usr = "아이디"
pwd = "비밀번호"
query = com_list

# step3.크롬드라이버 실행 및 잡플래닛 로그인
driver = webdriver.Chrome("/Users/ijunhui/Downloads/ubion/프로젝트2/chromedriver 2")

driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")
time.sleep(5)

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)

login_pwd = driver.find_element_by_css_selector("input#user_password")
login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)


list_allstar = []
list_company = []
list_date =[]
list_stars = []
list_summery = []
list_merit = []
list_disadvantages = []
list_managers =[]
list_score = []


# 회사
for i in query[207:]:
    try:
        # 검색창에 기업명 입력
        search_query = driver.find_element_by_css_selector("input#search_bar_search_query")
        time.sleep(3)
        search_query.clear()
        search_query.send_keys(i)
        search_query.send_keys(Keys.RETURN)
        time.sleep(3)
    
        #가장 먼저 나오는 회사 클릭 // 정확한 검색을 위해서 일치 하는 기업을 선택할 순 없을까... (여기서 가장 많은 기업들 선택 오류 발생)
        driver.find_element_by_css_selector("a.tit").click()
        time.sleep(1)

        try:
            time.sleep(1)
            # 팝업 삭제
            driver.find_element_by_css_selector("button.btn_close_x_ty1 ").click()
            time.sleep(3)

        except:
            pass

        time.sleep(3)
        # 페이지 계산 (전체 리뷰수 / 5(한페이지 최대 리뷰수)
        # 전체 리뷰 수 
        page = driver.find_element(By.XPATH,  "//*[@id='viewReviewsTitle']/span").text
        count_w = driver.find_elements_by_css_selector("h2.us_label")
        count = int(len(count_w))
        # max = math.ceil((int(page))/5)
        # if max > 50
        # 몇 페이지 까지
        print(math.ceil((int(page))/5)+1)
        for l in range(1, math.ceil((int(page))/5)+1): # 페이지 수 만큼 반복
            for j in range(1,6):
                
                # 전체 평점
                allstar = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div[2]/div/div/div[1]/div[2]/div[2]/div[1]/div/span").text

                try:
                    # 날짜
                    date = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[1]/span[8]").text
                    list_date.append(date)
                except:
                    try:
                        # 기업에 리뷰가 하나만 있을 시 경로가 다름
                        date = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section/div/div[1]/span[6]").text
                        list_date.append(date)
                    except:
                        pass
                try:
                    # 회사명 // ts2000과의 기업명이 서로 상이 함... (주)의 유무, 영어 TS <--> 티에스
                    com = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div[2]/div/div/div[1]/div[2]/div[1]/h1/a").text

                    # 요약
                    summery = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[2]/div/div[1]/h2").text

                    # 장점
                    merit = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[2]/div/dl/dd[1]/span").text

                    # 단점
                    disadvantages = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[2]/div/dl/dd[2]/span").text

                    # CEO에게
                    managers = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[2]/div/dl/dd[3]/span").text

                    # 평점 왜인지 모르겠지만 하니씩 밀렸던 평점 그냥 일일히 찾아서 오는 방법으로 변경, style 호출 방법 
                    stars = driver.find_element(By.XPATH, "//*[@id='viewReviewsList']/div/div/div/section[" + str(j) + "]/div/div[2]/dl/dd[1]/div/div").get_attribute("style")

                    if stars[7:9] == '20':
                        list_stars.append("1점")
                    elif stars[7:9] == '40':
                        list_stars.append("2점")
                    elif stars[7:9] == '60':
                        list_stars.append("3점")
                    elif stars[7:9] == '80':
                        list_stars.append("4점")
                    else:
                        list_stars.append("5점")

                    list_allstar.append(allstar)            
                    list_company.append(com)
                    list_summery.append(summery)
                    list_merit.append(merit)
                    list_disadvantages.append(disadvantages)
                    list_managers.append(managers)
                    
                except NoSuchElementException:
                    # 댓글 삭제 시 날짜는 존재하지만 나머지(요약, 장점, 단점, CEO에게)는 존재하지 않기 떄문에 예외 처리
                    list_allstar.append("날 괴롭힌 삭제된 댓글")
                    list_company.append(com)
                    list_summery.append("")
                    list_merit.append("")
                    list_disadvantages.append("")
                    list_managers.append("")
                    list_stars.append("")

                    # df = pd.DataFrame({'총점' : list_allstar, '회사' : list_company, '날짜' : list_date, '별점' : list_stars, '요약' : list_summery , '장점' : list_merit, '단점' : list_disadvantages, "경영진에게" : list_managers})        
                    # df.to_csv('./잡플레닛크롤링' + str(i) + '.csv', index=False)
            try:
                # 다음 페이지 클릭
                driver.find_element_by_css_selector("a.btn_pgnext").click()
                time.sleep(3)
            except:
                print(query, "없음")
                pass


    except:
        pass

    print(list_company)
    df_1 = pd.DataFrame({'총점' : list_allstar, '회사' : list_company, '날짜' : list_date, '별점' : list_stars, '요약' : list_summery , '장점' : list_merit, '단점' : list_disadvantages, "경영진에게" : list_managers})        
    df_1.to_csv('/Users/ijunhui/Downloads/ubion/프로젝트2/2_크롤링/플랫폼/정상/2017/잡플레닛/' + str(i) + '.csv', index=False)



51
['(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우', '(주)연우

In [99]:
df_1

,총점,회사,날짜,별점,요약,장점,단점,경영진에게
0,2.0,(주)제닉,2022. 09,3점,"""모두가 열심히 하면 다시 예전의 명성을 찾을 수 있지 않을까요?""",밥이 맛있다.\n칼퇴 가능하며 워라벨 최고!\n초봉이 높다,열심히 하는 사람들은 열심히 하는데 노는 사람들 또한 많음\n명절 상여금이 없다.....,모두가 다 열심히 하게끔 으쌰으쌰 해주셨으면 좋겠습니다.\n회사 제품도 좋고 명성도...
1,2.0,(주)제닉,2022. 09,2점,"""편하게 다닐 수 있는 회사! 비전 없음\n월루 임원들 많이 모이는 회사""",연차 자유럽게 쓸 수 있음\n칼퇴 보장\n밥 잘 나옴\n같이 일파는 동료들 다 친절...,무능한 팀장으로 인해 실무자 엄청 힘들게 함\n방향성 없음\n체계 안 잡힘\n하루종...,실무자 말을 잘 들었으면 좋겠음…\n실제 근무 상황 일도 파악 안 되고 윗사람들 다...
2,2.0,(주)제닉,2022. 08,3점,"""그냥 저냥 다닐만합니다\n엄청 추천하지는 않음..""",주5일에 주말에 쉰다는게 유일해보이는 장점..?\n밥도 먹을만합니다,"적은 임금, 성과급 없고 호봉제의 경우잔업없을경우 급여가 처참함",급여 올려줬으면 좋겠네요...\n명절에 떡값이라도
3,2.0,(주)제닉,2022. 08,2점,"""마스크 팩 제품에 대한 전문적인 지식 및 경험을 원한다면 추천""","연차 사용에 눈치보지 않고 자유로움, 식당이 있어 점심이 제공 됨, 대부분 괜찮은 ...","매출이 줄어서인지 공장 가동률이 예전같지 않아서 강제 휴무 등이 늘어나고 있었음 ,...",회사가 점 점 쪼그라들고 있는 듯한데... 전문가 투입으로 옛 제닉의 명성을 찾아주세요.
4,2.0,(주)제닉,2022. 07,3점,"""일하는 사람만 일하고 일안하는 사람은 편한곳""","연차를 눈치 안보고 쓸수 있음, 식비가 들지 않음 칼퇴가능함",일안하는 사람들이 너무 많아서 일하는 사람만 일하게 됨\n무능력한 사람 천지라 제대...,직원들의 불만을 들어주고 월급만 축내는 사람들은 내보냈으면 좋겠음
...,...,...,...,...,...,...,...,...
740,2.0,(주)모베이스전자,2020. 12,2점,"""시스템은 대기업 및 중견기업이며, 사내문화나 조직운영은 자동차부품 제조업 특성상 ...","중견, 대기업에서 진행되는 프로세스 및 업무역량을 배울수 있음","자동차 제조업 특성인 보수적인 조직문화가 있으며, 매년 노동조합과의 갈등으로 발생하...","사내에 바른 조직문화가 필요하며, 올바른 경영시스템 및 인재경영이 필요함"
741,2.0,(주)모베이스전자,2020. 12,2점,"""매출은 많은데,영업이익은 폭망. 현대기아가 메인. 급변하는 자동차 시대에 살아 남...",다양한 아이템을 하고 있음. 업무 시스템은 잘 갖춰줘 있다고 생각은 함,다품종 아이템을 하고 있어 개인당 업무량이 많고 집중도가 떨어짐. 낮은 연봉 테이블...,없음. 잘했다면 매번 팔리진 않았겠지. 기대도 안함\n좋은 곳에 팔렸음 좋겠다
742,2.0,(주)모베이스전자,2020. 12,1점,"""어디에가서도 버틸수있는 멘탈을 가지고 싶은자 모베로오라""","자녀 양육비 지원, 학자금지원이 있음 그마저도 근래에는 밀린적 있음 강성노조라 대리...",너무 많아서 나열하기 힘듬 회사에 불리한 모든것들은 사전공지 절대안함 이해할수 없는...,말로만 소리치지마시고 직원들 얘기좀 귀열고 들으셧으면 합니다.
743,2.0,(주)모베이스전자,2020. 12,1점,"""본인 돈으로 출장 다녀야 하는것도 모자라서 직원 퇴직금까지 미뤄가면서 어떻게든 회...",연구소가 그나마 평촌에 있다는 점 빼고는 장점을 눈을 씻고 찾아봐도 볼 수 없음.\...,퇴직한지 한달 넘게 퇴직금 지급이 안되었음.\n본인 뿐만 아니라 다른 퇴직자들한테도...,이런식으로 직원들한테 피해줘놓고 직원들한테 애사심을 바란다는게 정상적이라고 생각하나...


In [101]:
df_1.to_csv('/Users/ijunhui/Downloads/ubion/프로젝트2/2_크롤링/플랫폼/정상/2017/잡플레닛/제닉_모베이스전자.csv', index=False)
